In [25]:
""" Ingest PCRGLOBWB timeseries data on Google Earth Engine
-------------------------------------------------------------------------------
This notebook will upload the geotiff files from the Google Cloud Storage to
the WRI/aqueduct earthengine bucket. 

Requirements:
    Authorize earthengine by running in your terminal: earthengine 
                                                       authenticate

    you need to have access to the WRI-Aquaduct (yep a Google employee made a
    typo) bucket to ingest the data. Rutger can grant access to write to this 
    folder. 

    Have access to the Google Cloud Storage Bucker

Make sure to set the project to Aqueduct30 by running
`gcloud config set project aqueduct30`

Code follows the Google for Python Styleguide. Exception are the scripts that 
use earth engine since this is camelCase instead of underscore.

Author: Rutger Hofste
Date: 20170802
Kernel: python27
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:    
    TESTING (Boolean) : Toggle Testing Mode.
    OVERWRITE (Boolean) : Overwrite old folder !CAUTION!
    SCRIPT_NAME (string) : Script name.
    GCS_BASE (string) : Google Cloud Storage namespace.
    EE_BASE (string) : Earth Engine folder to store the imageCollections
    OUTPUT_FILE_NAME (string) : File Name for a csv file containing the failed tasks. 
    S3_OUTPUT_PATH (string) : Amazon S3 Output path.

Returns:


"""

# Input Parameters
TESTING = 0
OVERWRITE = 0 # !CAUTION!
SCRIPT_NAME = "Y2017M08D02_RH_Ingest_GCS_EE_V02"
GCS_BASE = "gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V02/"
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V08"
OUTPUT_FILE_NAME = "df_errorsV01.csv"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/{}/output".format(SCRIPT_NAME)

In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M04D09 UTC 12:18


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
# Imports
import subprocess
import datetime
import os
import time
import re
import pandas as pd
from datetime import timedelta
import aqueduct3

In [4]:
# ETL

ec2_output_path = "/volumes/data/{}/output".format(SCRIPT_NAME)

if OVERWRITE:
    command = "earthengine rm -r {}".format(EE_BASE)
    print(command)
    subprocess.check_output(command,shell=True)

command = "earthengine create folder {}".format(EE_BASE)
print(command)
subprocess.check_output(command,shell=True)

earthengine rm -r projects/WRI-Aquaduct/PCRGlobWB20V08
earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V08


b''

In [5]:
# Functions

def split_key(key):
    """ Split key into dictionary
    -------------------------------------------------------------------------------
    WARNING: This function is dependant on the name convention of PCRGLOBWB
    Do not use with other keys
    
    Args:
        key (string) : key containing information about parameter, year month etc.
        
    Returns:
        out_dict (dictionary): Dictionary containing all information contained
                               in key.      

    """
    
    # will yield the root file code and extension of a set of keys
    prefix, extension = key.split(".")
    file_name = prefix.split("/")[-1]
    parameter = file_name[:-12]
    month = file_name[-2:] #can also do this with regular expressions if you like
    year = file_name[-7:-3]
    identifier = file_name[-11:-8]
    out_dict = {"file_name":file_name,"extension":extension,"parameter":parameter,"month":month,"year":year,"identifier":identifier}
    return out_dict

def split_parameter(parameter):
    """Split parameter 
    -------------------------------------------------------------------------------
    WARNING: This function is dependant on the name convention of PCRGLOBWB
    Do not use with other keys.
    
    Args:
        parameter (string) : parameter string.
    
    Returns:
        out_dict (dictionary) : dictionary containing all information contained
                                in parameter key.
    
    """
    
    values = re.split("_|-", parameter) #soilmoisture uses a hyphen instead of underscore between the years
    keys = ["geographic_range","temporal_range","indicator","temporal_resolution","units","spatial_resolution","temporal_range_min","temporal_range_max"]
    # ['global', 'historical', 'PDomWN', 'month', 'millionm3', '5min', '1960', '2014']
    out_dict = dict(zip(keys, values))
    out_dict["parameter"] = parameter
    return out_dict


def get_GCS_keys(GCS_BASE):
    """ get list of keys from Google Cloud Storage
    -------------------------------------------------------------------------------
    
    Args:
        GCS_BASE (string) : Google Cloud Storage namespace containing files.
        
    Returns:
        df (pd.DataFrame) : DataFrame with properties useful to Aqueduct. 
    
    """
    command = "/opt/google-cloud-sdk/bin/gsutil ls {}".format(GCS_BASE)
    keys = subprocess.check_output(command,shell=True)
    keys = keys.decode('UTF-8').splitlines()
    
    df = keys_to_df(keys)
    
    return df

def keys_to_df(keys):
    """ helper function for 'get_GCS_keys'
    
    Args:
        keys (list) : list of strings with keys.
        
    Returns:
        df (pd.DataFrame) : Pandas DataFrame with all relvant properties for
                            Aqueduct 3.0.
    """
    
    df = pd.DataFrame()
    i = 0
    for key in keys:
        i = i+1
        out_dict = split_key(key)
        df2 = pd.DataFrame(out_dict,index=[i])
        df = df.append(df2)    
    return df

In [6]:
# Script
df = get_GCS_keys(GCS_BASE)
df.shape

(9290, 6)

In [7]:
# Create ImageCollections
parameters = df.parameter.unique()
for parameter in parameters:
    ic_id = EE_BASE + "/" + parameter
    command, result = aqueduct3.create_imageCollection(ic_id)
    print(command,result)

earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PDomWN_month_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PDomWN_year_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PDomWW_month_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PDomWW_year_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PIndWN_month_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PIndWN_year_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PIndWW_month_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V08/global_historical_PIndWW_year_millionm3_5min_19

In [8]:
# Prepare Dataframe
df_parameter = pd.DataFrame()
i = 0
for parameter in parameters:
    i = i+1
    out_dict_parameter = split_parameter(parameter)
    df_parameter2 = pd.DataFrame(out_dict_parameter,index=[i])
    df_parameter = df_parameter.append(df_parameter2)   
    

In [9]:
df_parameter.shape

(25, 9)

In [10]:
df_complete = df.merge(df_parameter,how='left',left_on='parameter',right_on='parameter')

Adding NoData value, ingested_by and exportdescription

In [11]:
df_complete["nodata_value"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"] + "_" + df_complete["temporal_resolution"]+"Y"+df_complete["year"]+"M"+df_complete["month"]
df_complete["script_used"] = SCRIPT_NAME
df_complete = df_complete.apply(pd.to_numeric, errors='ignore')

In [12]:
df_complete.head()

,extension,file_name,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata_value,ingested_by,exportdescription,script_used
0,tif,global_historical_PDomWN_month_millionm3_5min_...,0,1,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M01,Y2017M08D02_RH_Ingest_GCS_EE_V02
1,tif,global_historical_PDomWN_month_millionm3_5min_...,1,2,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M02,Y2017M08D02_RH_Ingest_GCS_EE_V02
2,tif,global_historical_PDomWN_month_millionm3_5min_...,2,3,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M03,Y2017M08D02_RH_Ingest_GCS_EE_V02
3,tif,global_historical_PDomWN_month_millionm3_5min_...,3,4,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M04,Y2017M08D02_RH_Ingest_GCS_EE_V02
4,tif,global_historical_PDomWN_month_millionm3_5min_...,4,5,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M05,Y2017M08D02_RH_Ingest_GCS_EE_V02


In [13]:
df_complete.tail()

,extension,file_name,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata_value,ingested_by,exportdescription,script_used
9285,tif,global_historical_soilmoisture_month_meter_5mi...,679,8,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M08,Y2017M08D02_RH_Ingest_GCS_EE_V02
9286,tif,global_historical_soilmoisture_month_meter_5mi...,680,9,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M09,Y2017M08D02_RH_Ingest_GCS_EE_V02
9287,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M10,Y2017M08D02_RH_Ingest_GCS_EE_V02
9288,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M11,Y2017M08D02_RH_Ingest_GCS_EE_V02
9289,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M12,Y2017M08D02_RH_Ingest_GCS_EE_V02


In [14]:
list(df_complete.columns.values)

['extension',
 'file_name',
 'identifier',
 'month',
 'parameter',
 'year',
 'geographic_range',
 'indicator',
 'spatial_resolution',
 'temporal_range',
 'temporal_range_max',
 'temporal_range_min',
 'temporal_resolution',
 'units',
 'nodata_value',
 'ingested_by',
 'exportdescription',
 'script_used']

In [15]:
if TESTING:
    df_complete = df_complete[1:3]

In [16]:
df_errors = pd.DataFrame()
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"%.2f" %((index/df_complete.shape[0])*100), "elapsed: ", str(timedelta(seconds=elapsed_time)))
    
    geotiff_gcs_path = GCS_BASE + row.file_name + "." + row.extension
    output_ee_asset_id = EE_BASE +"/"+ row.parameter + "/" + row.file_name
    properties = row.to_dict()
    
    df_errors2 = aqueduct3.upload_geotiff_to_EE_imageCollection(geotiff_gcs_path, output_ee_asset_id, properties,index)
    df_errors = df_errors.append(df_errors2)

0 0.00 elapsed:  0:00:00.003813
1 0.01 elapsed:  0:00:03.145800
2 0.02 elapsed:  0:00:06.274937
3 0.03 elapsed:  0:00:09.451509
4 0.04 elapsed:  0:00:12.578020
5 0.05 elapsed:  0:00:15.724248
6 0.06 elapsed:  0:00:18.881264
7 0.08 elapsed:  0:00:21.635190
8 0.09 elapsed:  0:00:24.855043
9 0.10 elapsed:  0:00:27.987987
10 0.11 elapsed:  0:00:31.141404
11 0.12 elapsed:  0:00:34.374884
12 0.13 elapsed:  0:00:37.510258
13 0.14 elapsed:  0:00:40.649844
14 0.15 elapsed:  0:00:43.814646
15 0.16 elapsed:  0:00:47.049090
16 0.17 elapsed:  0:00:50.251041
17 0.18 elapsed:  0:00:53.218890
18 0.19 elapsed:  0:00:56.265232
19 0.20 elapsed:  0:00:58.850168
20 0.22 elapsed:  0:01:01.881372
21 0.23 elapsed:  0:01:04.484252
22 0.24 elapsed:  0:01:07.593973
23 0.25 elapsed:  0:01:10.912996
24 0.26 elapsed:  0:01:14.096498
25 0.27 elapsed:  0:01:17.188025
26 0.28 elapsed:  0:01:20.240081
27 0.29 elapsed:  0:01:23.306115
28 0.30 elapsed:  0:01:26.376824
29 0.31 elapsed:  0:01:29.471426
30 0.32 elapsed:  0:

245 2.64 elapsed:  0:12:26.832767
246 2.65 elapsed:  0:12:29.351429
247 2.66 elapsed:  0:12:32.313870
248 2.67 elapsed:  0:12:35.340044
249 2.68 elapsed:  0:12:38.362208
250 2.69 elapsed:  0:12:41.439790
251 2.70 elapsed:  0:12:43.497428
252 2.71 elapsed:  0:12:46.538449
253 2.72 elapsed:  0:12:49.505823
254 2.73 elapsed:  0:12:52.541061
255 2.74 elapsed:  0:12:55.524099
256 2.76 elapsed:  0:12:58.603656
257 2.77 elapsed:  0:13:01.628689
258 2.78 elapsed:  0:13:04.687549
259 2.79 elapsed:  0:13:07.707254
260 2.80 elapsed:  0:13:10.662138
261 2.81 elapsed:  0:13:13.208371
262 2.82 elapsed:  0:13:16.216980
263 2.83 elapsed:  0:13:19.315651
264 2.84 elapsed:  0:13:21.885189
265 2.85 elapsed:  0:13:24.961110
266 2.86 elapsed:  0:13:27.999148
267 2.87 elapsed:  0:13:30.614256
268 2.88 elapsed:  0:13:33.649443
269 2.90 elapsed:  0:13:36.627459
270 2.91 elapsed:  0:13:39.208979
271 2.92 elapsed:  0:13:42.487540
272 2.93 elapsed:  0:13:45.680098
273 2.94 elapsed:  0:13:48.838964
274 2.95 elaps

486 5.23 elapsed:  0:24:51.339235
487 5.24 elapsed:  0:24:54.611188
488 5.25 elapsed:  0:24:57.895413
489 5.26 elapsed:  0:25:00.683342
490 5.27 elapsed:  0:25:03.869516
491 5.29 elapsed:  0:25:06.674750
492 5.30 elapsed:  0:25:10.178781
493 5.31 elapsed:  0:25:13.469121
494 5.32 elapsed:  0:25:16.208289
495 5.33 elapsed:  0:25:19.545670
496 5.34 elapsed:  0:25:22.745459
497 5.35 elapsed:  0:25:25.497416
498 5.36 elapsed:  0:25:28.155983
499 5.37 elapsed:  0:25:31.514352
500 5.38 elapsed:  0:25:34.384436
501 5.39 elapsed:  0:25:37.692690
502 5.40 elapsed:  0:25:40.021988
503 5.41 elapsed:  0:25:43.331546
504 5.43 elapsed:  0:25:46.153786
505 5.44 elapsed:  0:25:49.430766
506 5.45 elapsed:  0:25:52.679598
507 5.46 elapsed:  0:25:55.952520
508 5.47 elapsed:  0:25:58.743873
509 5.48 elapsed:  0:26:01.933095
510 5.49 elapsed:  0:26:05.019578
511 5.50 elapsed:  0:26:08.260413
512 5.51 elapsed:  0:26:11.369221
513 5.52 elapsed:  0:26:14.070363
514 5.53 elapsed:  0:26:17.181673
515 5.54 elaps

727 7.83 elapsed:  0:37:19.674437
728 7.84 elapsed:  0:37:22.757539
729 7.85 elapsed:  0:37:25.857872
730 7.86 elapsed:  0:37:28.975154
731 7.87 elapsed:  0:37:32.261700
732 7.88 elapsed:  0:37:35.404889
733 7.89 elapsed:  0:37:38.550065
734 7.90 elapsed:  0:37:41.682989
735 7.91 elapsed:  0:37:44.788491
736 7.92 elapsed:  0:37:47.922789
737 7.93 elapsed:  0:37:51.025311
738 7.94 elapsed:  0:37:54.098188
739 7.95 elapsed:  0:37:57.236024
740 7.97 elapsed:  0:38:00.320852
741 7.98 elapsed:  0:38:03.469121
742 7.99 elapsed:  0:38:06.559300
743 8.00 elapsed:  0:38:09.640383
744 8.01 elapsed:  0:38:12.769325
745 8.02 elapsed:  0:38:15.842842
746 8.03 elapsed:  0:38:18.928402
747 8.04 elapsed:  0:38:22.086070
748 8.05 elapsed:  0:38:25.168260
749 8.06 elapsed:  0:38:28.294362
750 8.07 elapsed:  0:38:31.482421
751 8.08 elapsed:  0:38:34.589100
752 8.09 elapsed:  0:38:37.697006
753 8.11 elapsed:  0:38:40.806597
754 8.12 elapsed:  0:38:43.943902
755 8.13 elapsed:  0:38:47.012624
756 8.14 elaps

967 10.41 elapsed:  0:49:44.901223
968 10.42 elapsed:  0:49:47.960159
969 10.43 elapsed:  0:49:51.006060
970 10.44 elapsed:  0:49:54.080335
971 10.45 elapsed:  0:49:57.144026
972 10.46 elapsed:  0:50:00.179380
973 10.47 elapsed:  0:50:03.238206
974 10.48 elapsed:  0:50:06.316267
975 10.50 elapsed:  0:50:09.407763
976 10.51 elapsed:  0:50:12.576752
977 10.52 elapsed:  0:50:15.654939
978 10.53 elapsed:  0:50:18.751776
979 10.54 elapsed:  0:50:21.768229
980 10.55 elapsed:  0:50:24.821636
981 10.56 elapsed:  0:50:27.982328
982 10.57 elapsed:  0:50:31.042391
983 10.58 elapsed:  0:50:34.096612
984 10.59 elapsed:  0:50:37.258849
985 10.60 elapsed:  0:50:40.318072
986 10.61 elapsed:  0:50:43.475934
987 10.62 elapsed:  0:50:46.554360
988 10.64 elapsed:  0:50:49.614085
989 10.65 elapsed:  0:50:52.627347
990 10.66 elapsed:  0:50:55.695503
991 10.67 elapsed:  0:50:58.743111
992 10.68 elapsed:  0:51:02.042842
993 10.69 elapsed:  0:51:05.018994
994 10.70 elapsed:  0:51:08.090121
995 10.71 elapsed:  

...

8264 88.96 elapsed:  7:08:21.909866
8265 88.97 elapsed:  7:08:24.937031
8266 88.98 elapsed:  7:08:28.041388
8267 88.99 elapsed:  7:08:31.093067
8268 89.00 elapsed:  7:08:34.129350
8269 89.01 elapsed:  7:08:37.200763
8270 89.02 elapsed:  7:08:40.197611
8271 89.03 elapsed:  7:08:43.201569
8272 89.04 elapsed:  7:08:46.227775
8273 89.05 elapsed:  7:08:49.293885
8274 89.06 elapsed:  7:08:52.399549
8275 89.07 elapsed:  7:08:55.557959
8276 89.09 elapsed:  7:08:59.017996
8277 89.10 elapsed:  7:09:02.325593
8278 89.11 elapsed:  7:09:05.513074
8279 89.12 elapsed:  7:09:08.732942
8280 89.13 elapsed:  7:09:11.994531
8281 89.14 elapsed:  7:09:15.469455
8282 89.15 elapsed:  7:09:18.244799
8283 89.16 elapsed:  7:09:21.448268
8284 89.17 elapsed:  7:09:24.612598
8285 89.18 elapsed:  7:09:28.013293
8286 89.19 elapsed:  7:09:31.419391
8287 89.20 elapsed:  7:09:34.617410
8288 89.21 elapsed:  7:09:37.770931
8289 89.22 elapsed:  7:09:41.237020
8290 89.24 elapsed:  7:09:44.423523
8291 89.25 elapsed:  7:09:47

8492 91.41 elapsed:  7:20:21.106070
8493 91.42 elapsed:  7:20:24.189415
8494 91.43 elapsed:  7:20:27.315002
8495 91.44 elapsed:  7:20:30.380805
8496 91.45 elapsed:  7:20:33.526646
8497 91.46 elapsed:  7:20:36.631178
8498 91.47 elapsed:  7:20:39.743492
8499 91.49 elapsed:  7:20:42.380154
8500 91.50 elapsed:  7:20:45.455162
8501 91.51 elapsed:  7:20:48.531757
8502 91.52 elapsed:  7:20:51.647293
8503 91.53 elapsed:  7:20:54.760648
8504 91.54 elapsed:  7:20:57.901223
8505 91.55 elapsed:  7:21:01.064496
8506 91.56 elapsed:  7:21:04.186883
8507 91.57 elapsed:  7:21:06.770255
8508 91.58 elapsed:  7:21:09.882909
8509 91.59 elapsed:  7:21:12.988960
8510 91.60 elapsed:  7:21:16.152671
8511 91.61 elapsed:  7:21:19.294205
8512 91.63 elapsed:  7:21:22.388944
8513 91.64 elapsed:  7:21:25.524936
8514 91.65 elapsed:  7:21:28.645025
8515 91.66 elapsed:  7:21:31.722036
8516 91.67 elapsed:  7:21:34.754262
8517 91.68 elapsed:  7:21:37.834040
8518 91.69 elapsed:  7:21:40.854751
8519 91.70 elapsed:  7:21:43

8720 93.86 elapsed:  7:32:21.611261
8721 93.88 elapsed:  7:32:25.041187
8722 93.89 elapsed:  7:32:28.437846
8723 93.90 elapsed:  7:32:31.851946
8724 93.91 elapsed:  7:32:35.152822
8725 93.92 elapsed:  7:32:38.398378
8726 93.93 elapsed:  7:32:41.772758
8727 93.94 elapsed:  7:32:45.094598
8728 93.95 elapsed:  7:32:48.535638
8729 93.96 elapsed:  7:32:51.892698
8730 93.97 elapsed:  7:32:55.275349
8731 93.98 elapsed:  7:32:58.668166
8732 93.99 elapsed:  7:33:01.885197
8733 94.00 elapsed:  7:33:04.809796
8734 94.02 elapsed:  7:33:08.121323
8735 94.03 elapsed:  7:33:11.029875
8736 94.04 elapsed:  7:33:14.401464
8737 94.05 elapsed:  7:33:16.750476
8738 94.06 elapsed:  7:33:19.649828
8739 94.07 elapsed:  7:33:22.915271
8740 94.08 elapsed:  7:33:25.836674
8741 94.09 elapsed:  7:33:28.749954
8742 94.10 elapsed:  7:33:31.643824
8743 94.11 elapsed:  7:33:34.894927
8744 94.12 elapsed:  7:33:37.805350
8745 94.13 elapsed:  7:33:40.655999
8746 94.14 elapsed:  7:33:43.916227
8747 94.16 elapsed:  7:33:47

8948 96.32 elapsed:  7:44:19.315109
8949 96.33 elapsed:  7:44:22.362542
8950 96.34 elapsed:  7:44:25.437587
8951 96.35 elapsed:  7:44:27.971486
8952 96.36 elapsed:  7:44:30.597424
8953 96.37 elapsed:  7:44:33.591540
8954 96.38 elapsed:  7:44:36.640277
8955 96.39 elapsed:  7:44:39.675037
8956 96.40 elapsed:  7:44:42.767527
8957 96.42 elapsed:  7:44:45.826969
8958 96.43 elapsed:  7:44:48.839215
8959 96.44 elapsed:  7:44:51.832440
8960 96.45 elapsed:  7:44:54.831723
8961 96.46 elapsed:  7:44:57.949662
8962 96.47 elapsed:  7:45:00.644246
8963 96.48 elapsed:  7:45:03.855894
8964 96.49 elapsed:  7:45:07.060315
8965 96.50 elapsed:  7:45:09.908760
8966 96.51 elapsed:  7:45:13.182526
8967 96.52 elapsed:  7:45:16.569605
8968 96.53 elapsed:  7:45:19.931485
8969 96.54 elapsed:  7:45:23.258253
8970 96.56 elapsed:  7:45:26.461843
8971 96.57 elapsed:  7:45:29.728929
8972 96.58 elapsed:  7:45:33.030798
8973 96.59 elapsed:  7:45:36.404408
8974 96.60 elapsed:  7:45:39.239790
8975 96.61 elapsed:  7:45:42

9176 98.77 elapsed:  7:56:05.160419
9177 98.78 elapsed:  7:56:08.463620
9178 98.79 elapsed:  7:56:11.300160
9179 98.81 elapsed:  7:56:14.557139
9180 98.82 elapsed:  7:56:17.873264
9181 98.83 elapsed:  7:56:21.177902
9182 98.84 elapsed:  7:56:24.534073
9183 98.85 elapsed:  7:56:27.312453
9184 98.86 elapsed:  7:56:30.544774
9185 98.87 elapsed:  7:56:33.762052
9186 98.88 elapsed:  7:56:37.064357
9187 98.89 elapsed:  7:56:40.291464
9188 98.90 elapsed:  7:56:43.420160
9189 98.91 elapsed:  7:56:46.114751
9190 98.92 elapsed:  7:56:49.255926
9191 98.93 elapsed:  7:56:52.444487
9192 98.95 elapsed:  7:56:55.603453
9193 98.96 elapsed:  7:56:58.799680
9194 98.97 elapsed:  7:57:01.973967
9195 98.98 elapsed:  7:57:05.079447
9196 98.99 elapsed:  7:57:08.130367
9197 99.00 elapsed:  7:57:11.117291
9198 99.01 elapsed:  7:57:14.101288
9199 99.02 elapsed:  7:57:17.191566
9200 99.03 elapsed:  7:57:19.732042
9201 99.04 elapsed:  7:57:22.724925
9202 99.05 elapsed:  7:57:25.813136
9203 99.06 elapsed:  7:57:28

In [17]:
!mkdir -p {ec2_output_path}

In [18]:
df_errors.to_csv("{}/{}".format(ec2_output_path,OUTPUT_FILE_NAME))

In [19]:
!aws s3 cp  {ec2_output_path} {S3_OUTPUT_PATH} --recursive

upload failed: ../../../../data/Y2017M08D02_RH_Ingest_GCS_EE_V02/output/df_errorsV01.csv to s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Ingest_GCS_EE_V02/output/df_errorsV01.csv Unable to locate credentials


Retry the ones with errors

In [20]:
df_retry = df_errors.loc[df_errors['error'] != 0]

In [21]:
for index, row in df_retry.iterrows():
    response = subprocess.check_output(row.command, shell=True)
    

In [22]:
uniques = df_errors["error"].unique()

In [24]:
df_retry

,command,error,response


In [23]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

8:04:29.006024
